In [1]:
# !pip -q install sentence-transformers faiss-cpu gradio pypdf transformers accelerate
# import torch, numpy as np, textwrap, re, os
# from sentence_transformers import SentenceTransformer
# import faiss

# Tell Transformers not to load TensorFlow parts
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

# Install compatible versions
!pip -q install "sentence-transformers==2.7.0" "transformers==4.41.2" \
                 "faiss-cpu==1.7.4" "accelerate==0.33.0" "pypdf==4.2.0" "gradio==4.44.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-cpu==1.7.4 (from versions: 1.8.0, 1.8.0.post1, 1.9.0, 1.9.0.post1, 1.10.0, 1.11.0, 1.11.0.post1, 1.12.0)
ERROR: No matching distribution found for faiss-cpu==1.7.4


In [2]:
import numpy as np, textwrap, re, os
from sentence_transformers import SentenceTransformer
import faiss

In [3]:
# # OPTION A: paste a long string (you can replace this with your notes)
raw_text = """
# Transformers are neural networks that use self-attention to read all tokens at once...
# (You can paste your notes, policy docs, or any study material here)
# """

# OPTION B: upload a PDF and extract text
from google.colab import files
from pypdf import PdfReader

use_pdf = True  # set True if you want to upload a PDF

if use_pdf:
    uploaded = files.upload()  # choose a PDF file
    pdf_name = list(uploaded.keys())[0]
    pages = []
    for p in PdfReader(pdf_name).pages:
        pages.append(p.extract_text() or "")
    raw_text = "\n".join(pages)

print("Characters loaded:", len(raw_text))

Saving AI_RAG_Study_Guide.pdf to AI_RAG_Study_Guide (1).pdf
Characters loaded: 4298


In [5]:
def clean_text(t):
    t = re.sub(r'\s+', ' ', t).strip()
    return t

def make_chunks(t, chunk_size=500, overlap=100): # next paet starts at 400th word so that the model has context
    words = clean_text(t).split()
    chunks, i = [], 0
    while i < len(words):
        chunk = " ".join(words[i:i+chunk_size])
        if chunk:
            chunks.append(chunk)
        i += chunk_size - overlap
    return chunks

chunks = make_chunks(raw_text, chunk_size=180, overlap=40)  # small chunks for demo
print(f"Total chunks: {len(chunks)}")
chunks[:2]

Total chunks: 5


['Transformer Basics A transformer is a neural network that uses self-attention so every token can look at every other token at once. Key ideas: self-attention (Q, K, V), multi-head attention, feed-forward layers, residual connections, layer normalization, positional encodings. Benefits: parallel training (faster than RNNs), long-range dependence modeling, great scaling. Encoder models (BERT) are strong for classification and retrieval. Decoder models (GPT) are strong for generation. Encoder-decoder models (T5) are strong for translation and text-to-text tasks. Embeddings (Meaning as Vectors) An embedding maps text (or images/audio) to a dense vector where semantic closeness geometric closeness. Cosine similarity or dot-product measure closeness. Sentence/paragraph embeddings (e.g., MiniLM, mpnet) are used for semantic search, clustering, deduplication, and retrieval. Normalize embeddings for cosine. Good chunking and clean text improve retrieval quality. Embeddings are the backbone of

In [6]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)  # uses GPU if available

# Create normalized embeddings (cosine similarity = dot product on unit vectors)
emb = embedder.encode(chunks, convert_to_numpy=True, normalize_embeddings=True)
emb.shape

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(5, 384)

In [7]:
d = emb.shape[1]  # embedding dimension
index = faiss.IndexFlatIP(d)   # Inner Product (use with normalized embeddings = cosine) #faiss is lib let you find index in embeddings
index.add(emb)                 # add all chunk vectors
print("Vectors in index:", index.ntotal)

Vectors in index: 5


In [8]:
def search(query, k=3):
    q = embedder.encode([query], normalize_embeddings=True)
    scores, ids = index.search(q, k)     # scores shape: (1,k)
    results = []
    for score, idx in zip(scores[0], ids[0]): # Score is basically how close the numric entry of question and answer are: more score - exact answer
        results.append((float(score), idx, chunks[idx]))
    return results

# try it
for s, i, txt in search("What are transformers?", k=3):
    print("\nScore:", round(s, 3))
    print(textwrap.fill(txt, width=90))


Score: 0.314
Transformer Basics A transformer is a neural network that uses self-attention so every
token can look at every other token at once. Key ideas: self-attention (Q, K, V), multi-
head attention, feed-forward layers, residual connections, layer normalization, positional
encodings. Benefits: parallel training (faster than RNNs), long-range dependence modeling,
great scaling. Encoder models (BERT) are strong for classification and retrieval. Decoder
models (GPT) are strong for generation. Encoder-decoder models (T5) are strong for
translation and text-to-text tasks. Embeddings (Meaning as Vectors) An embedding maps text
(or images/audio) to a dense vector where semantic closeness geometric closeness. Cosine
similarity or dot-product measure closeness. Sentence/paragraph embeddings (e.g., MiniLM,
mpnet) are used for semantic search, clustering, deduplication, and retrieval. Normalize
embeddings for cosine. Good chunking and clean text improve retrieval quality. Embeddings
are th

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

gen_name = "google/flan-t5-small"

# tokenizer + model
tok = AutoTokenizer.from_pretrained(gen_name)  # pdf - broken down into chunks, question asked - gets converted into tokens and then in numeric form , similar numeric indices are found
# and the matching ones with scores are returned

device = "cuda" if torch.cuda.is_available() else "cpu"
# (optional: use fp16 on GPU for speed/memory)
dtype = torch.float16 if device == "cuda" else torch.float32

gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_name, torch_dtype=dtype).to(device)

def answer_with_context(question, k=3, max_new_tokens=128):
    # uses your earlier `search()` function
    hits = search(question, k=k)
    context = "\n\n".join([f"[chunk {i}] {txt}" for _, i, txt in hits])

    prompt = f"""Use the context to answer the question briefly.
If the answer isn't in the context, say you don't know.

Question:
{question}

Context:
{context}
"""
    inputs = tok(prompt, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        out = gen_model.generate(**inputs, max_new_tokens=max_new_tokens)
    ans = tok.decode(out[0], skip_special_tokens=True)
    return ans, hits

# quick test
ans, hits = answer_with_context("Explain self-attention in simple words.", k=3)
print("Answer:", ans)
print("Citations:", [i for _, i, _ in hits])

`torch_dtype` is deprecated! Use `dtype` instead!


Answer: [chunk 0]
Citations: [np.int64(0), np.int64(3), np.int64(2)]


In [10]:
import gradio as gr

def ask_fn(question):
    ans, hits = answer_with_context(question, k=3)
    cited = "\n\n".join([f"(score={s:.3f}) [chunk {i}] {textwrap.fill(txt[:350]+'...', 85)}" for s,i,txt in hits])
    return ans, cited

demo = gr.Interface(
    fn=ask_fn,
    inputs=gr.Textbox(lines=2, placeholder="Ask something about your text..."),
    outputs=[gr.Textbox(label="Answer"),
             gr.Textbox(label="Top-3 supporting chunks")],
    title="Mini Semantic Search + RAG",
    description="Ask questions about your pasted text or PDF. The model retrieves top chunks and (optionally) writes a short answer."
)

demo.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>